# Projet de programmation

## Introduction <a name="intro"></a>

Il faut récupèrer le projet avec : 

    git clone https://github.com/gwatkinson/projet-python-twitter.git

Puis, il faut créer le fichier `projet/credentials.py`, qui contient les clés de l'API de Twitter.

Dans le format suivant :

```python
credentials = {
    "consumer_key": "XXXXXXX",
    "consumer_secret": "XXXXXXX",
    "access_token": "XXXXXXX",
    "access_token_secret": "XXXXXXX",
}
```

Finalement, il suffit d'executer les cellules de ce notebook dans l'ordre.

## Table des matières

* [Introduction](#intro)
* [1)Récupération des données](#data)
* [2)Modélisation](#model)
    * [a.Prepocessing](#process)
    * [b.Clustering](#cluster)
* [3)Visualisation](#visu)
    * [a.Nuages de mots](#cloud)
    * [b.Carte interactive](#map)
* [Conclusion](#conc)
* [Annexes](#annex)

## 1) Récupération des données <a name="data"></a>



Nous avons utilisé l'**API** de Twitter pour récupérer les nouveaux tweets publiés sur Twitter, la nuit du 3 au 4 Novembre 2020 (la nuit de l'éléction américaine). Nous avons seulement récupérer les tweets qui contennaient certains mots :

In [2]:
# Liste 3 sur Trump et Biden uniquement
liste_3 = [
    "biden",
    "trump",
    "JoeBiden",
    "realDonaldTrump",
]

# Liste 4 sur le thème 'vote'
liste_4 = [
    "iwillvote",
    "govote",
    "uselection",
    "vote",
]

# Liste 5 sur le thème 'election'
liste_5 = [
    "uselection",
    "president",
    "presidentialelection",
    "presidential",
    "electionnight",
]

Pour cela, nous avons utilisé le module python `tweepy` ainsi que les fonctions codées dans le module `streaming` (voir la documentation pour plus d'information sur [`start_stream`](https://gwatkinson.github.io/projet-python-twitter/streaming.html#projet_python_twitter.streaming.start_stream)). Voici un exemple d'utilisation du code que nous avons écrit :

In [3]:
import projet.streaming as stream                           # Contient les fonctions pour le streaming
import projet.listes_mots as listes                         # Contient les listes de mots
import projet._credentials as cred                          # Contient les clés d'authentification à l'API

credentials = stream.CredentialsClass(cred.credentials)     # Pour se connecter à l'API (il faut le fichier projet/_credentials.py)

stream.start_stream(
    credentials=credentials,
    liste_mots=listes.liste_3,                              # Liste des mots à tracker (voir `projet.listes_mots`)
    nb=200,                                                 # Nombre de tweets à recupérer
    # timeout=10/3600,                                        # Durée du stream
    fprefix="exemple_liste_3",                              # À modifier en fonction de la liste selectionnée
    path="./data/json/",                                    # À modifier selon l'utilisateur (doit finir par "/" ou "\")
    verbose=True,
)

Début du stream
Progress: [------------------->] 100 %
Les 200 tweets ont été récupérés.
Le stream a duré : 0.0h
Fin du stream


Un fichier `json` a été créé dans `data/json/`.

Pour voir à quoi ressemble les données :

In [4]:
import glob
import json
import pandas as pd

path = glob.glob("data\\json\\exemple_liste_3*.json")[-1]  # On récupère le dernier fichier exemple crée
print("On regarde le fichier : "+path+"\n")

tweets_list = []
with open(path, "r") as fh:
    file = fh.read().split("\n")
    for line in file:
        if line:
            tweets_list.append(json.loads(line))

print("Le premier tweet :")
print(tweets_list[0])


On regarde le fichier : data\json\exemple_liste_3_20201210-171541.json

Le premier tweet :
{'created_at': 'Thu Dec 10 16:15:36 +0000 2020', 'id': 1337068459008507908, 'id_str': '1337068459008507908', 'text': 'RT @JohnJosephRath4: @TheOfficerTatum @burt_j @realDonaldTrump Me as well!!', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 132292378, 'id_str': '132292378', 'name': 'Burt Jurgens', 'screen_name': 'burt_j', 'location': 'Portland, OR', 'url': 'https://www.youtube.com/channel/UCfsllJSXNwVb8mrGj-S7byA', 'description': 'Former marine and deputy sheriff, fake news researcher #Trump2020 #DigitalSoldiers #SemperFi #WalkAway #MAGA #WWG1WGA #AllGloryToGod #OathKeeper #MidnightRider', 'translator_type': 'none', 'protected': False, 'verified': False,

Il s'agit du format `json`. Il est difficile de voir les variable comme cela. On peut créer une `dataframe pandas` pour mieux comprendre les données.

In [5]:
df_tweets = pd.DataFrame(tweets_list)
df_tweets.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,extended_tweet,extended_entities
0,Thu Dec 10 16:15:36 +0000 2020,1337068459008507908,1337068459008507908,RT @JohnJosephRath4: @TheOfficerTatum @burt_j ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607616936815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu Dec 10 16:15:36 +0000 2020,1337068458920448001,1337068458920448001,@realDonaldTrump @OANN 1:51. You have lost ag...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,1.337043e+18,1337042714924380166,25073877.0,25073877,...,en,1607616936794,"[23, 60]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thu Dec 10 16:15:36 +0000 2020,1337068458832515072,1337068458832515072,RT @SeanAKehoe: The only way President Trump’s...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1607616936773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu Dec 10 16:15:36 +0000 2020,1337068458740158469,1337068458740158469,RT @realDonaldTrump: Voter Fraud! https://t.co...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,fr,1607616936751,NaN,1.334962e+18,1334962273861951491,{'created_at': 'Fri Dec 04 20:46:23 +0000 2020...,"{'url': 'https://t.co/6mLqg6rmIZ', 'expanded':...",False,NaN,NaN
4,Thu Dec 10 16:15:36 +0000 2020,1337068459117617152,1337068459117617152,RT @PastorDScott: Why would Hunter Biden be un...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607616936841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print("Dimensions : ", df_tweets.shape)

Dimensions :  (200, 36)


In [7]:
print("Colonnes :\n")
for name in list(df_tweets):
    print(name)

Colonnes :

created_at
id
id_str
text
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
user
geo
coordinates
place
contributors
retweeted_status
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
entities
favorited
retweeted
filter_level
lang
timestamp_ms
display_text_range
quoted_status_id
quoted_status_id_str
quoted_status
quoted_status_permalink
possibly_sensitive
extended_tweet
extended_entities


## 2) Modélisation <a name="model"></a>

### a. Preprocessing <a name="process"></a>

Nous avons fait une fonction qui fait les étapes précédentes ainsi que des fonctions pour nettoyer les données. Elles sont dans le fichier `processing.py`.

In [8]:
import projet.processing as process                         # Contient les fonctions pour le processing de la dataframe

folder = "./data/json/"                                     # Pour écupèrer tous les fichiers json dans le dossier 'data/json/'

dirty_df = process.tweet_json_to_df(folder=folder, verbose=True)     # Convertit les json en dataframe pandas


La conversion des fichiers 'json' a commencé, cela peut prendre du temps
File 1/5: [------------------>] 100 %
File 2/5: [------------------>] 100 %
File 3/5: [------------------>] 100 %
File 4/5: [------------------>] 100 %
File 5/5: [------------------>] 100 %


Nous avons ainsi récupérer les fichiers dans `data/json/` dans la dataframe pandas `dirty_df`.

Elle ressemble à :

In [9]:

dirty_df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,possibly_sensitive,extended_tweet,extended_entities
0,Thu Dec 10 11:02:36 +0000 2020,1336989688167587848,1336989688167587848,RT @SebGorka: I’m telling President @realDonal...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1607598156383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu Dec 10 11:02:36 +0000 2020,1336989688154869760,1336989688154869760,RT @LilyMasonPhD: The fact that most GOP leade...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607598156380,1.336790e+18,1336790252963303425,{'created_at': 'Wed Dec 09 21:50:07 +0000 2020...,"{'url': 'https://t.co/W3VabzdvhV', 'expanded':...",NaN,NaN,NaN,NaN
2,Thu Dec 10 11:02:36 +0000 2020,1336989688565964801,1336989688565964801,RT @WalshFreedom: I love democracy.\n\nSo fuck...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607598156478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu Dec 10 11:02:36 +0000 2020,1336989688612220928,1336989688612220928,@Thinkingifree17 @llgraves @AngelaBelcamino @r...,"<a href=""http://twitter.com/download/android"" ...",False,1.336982e+18,1336981556540403712,8.482265e+17,848226546624200704,...,en,1607598156489,NaN,NaN,NaN,NaN,"[84, 128]",NaN,NaN,NaN
4,Thu Dec 10 11:02:36 +0000 2020,1336989688612212736,1336989688612212736,RT @jenniferatntd: So evil! And he admitted th...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1607598156489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On peut ensuite utiliser `clean_df` pour nettoyer la base de donnée en conservant seulement les informations qui nous interressent. On peut aussi utiliser une liste de `listes_variables` pour récupérer d'autres variables ou en ajouter dans l'option `extra` de `clean_df`.

In [10]:
from importlib import reload
reload(process)
clean_df = process.clean_df(dirty_df, verbose=True)

Le nettoyage a commencé
Progress: [------------------->] 100 %


In [52]:
clean_df.head()

,created_at,text,lang,quote_count,reply_count,retweet_count,favorite_count,user-id,user-description,user-verified,...,user-default_profile,user-lang,place-name,place-full_name,place-place_type,place-country_code,place-bounding_box,place-bounding_box-type,place-bounding_box-coordinates,entities
id,,,,,,,,,,,,,,,,,,,,,
1336989688167587848,2020-12-10 11:02:36+00:00,RT @SebGorka: I’m telling President @realDonal...,en,0,0,0,0,31351471,"NRA Conservative, Entrepreneur, Bitcoin & Cryp...",False,...,False,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
1336989688154869760,2020-12-10 11:02:36+00:00,RT @LilyMasonPhD: The fact that most GOP leade...,en,0,0,0,0,192026827,"I make things: sometimes awesome, sometimes no...",False,...,False,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
1336989688565964801,2020-12-10 11:02:36+00:00,RT @WalshFreedom: I love democracy.\n\nSo fuck...,en,0,0,0,0,202808641,"~ A dreamer, a learner, an imaginer",False,...,False,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
1336989688612220928,2020-12-10 11:02:36+00:00,@Thinkingifree17 @llgraves @AngelaBelcamino @r...,en,0,0,0,0,949059464,"🛡#Transgender, #Bi, #Poly 5 years HRT (04-07-1...",False,...,False,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
1336989688612212736,2020-12-10 11:02:36+00:00,RT @jenniferatntd: So evil! And he admitted th...,en,0,0,0,0,2531683835,"Mum to Ben, Jamie and my special baby girl Amy",False,...,True,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'hashtags': [{'text': 'COVID19', 'indices': [..."


In [11]:
list(clean_df)

['created_at',
 'text',
 'lang',
 'quote_count',
 'reply_count',
 'retweet_count',
 'favorite_count',
 'user-id',
 'user-name',
 'user-screen_name',
 'user-location',
 'user-description',
 'user-protected',
 'user-verified',
 'user-followers_count',
 'user-friends_count',
 'user-listed_count',
 'user-favourites_count',
 'user-statuses_count',
 'user-created_at',
 'user-contributors_enabled',
 'user-geo_enabled',
 'user-profile_background_color',
 'user-profile_sidebar_border_color',
 'user-profile_sidebar_fill_color',
 'user-profile_text_color',
 'user-profile_use_background_image',
 'user-default_profile',
 'user-following',
 'user-follow_request_sent',
 'user-notifications',
 'place-name',
 'place-full_name',
 'place-place_type',
 'place-country_code',
 'place-country',
 'place-bounding_box-type',
 'place-bounding_box-coordinates']

In [50]:
sum(clean_df['user-lang'].isnull())/clean_df.shape[0]

1.0

### b. Clustering <a name="cluster"></a>

## 3) Visualisation <a name="visu"></a>

### a. Nuages de mots <a name="cloud"></a>

### b. Carte interactive <a name="map"></a>

## Conclusion <a name="conc"></a>

## Annexes <a name="annex"></a>